In [96]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from gensim.models import word2vec
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer
from wordcloud import wordcloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob,Word
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from nltk.tokenize.toktok import ToktokTokenizer
from keras.preprocessing.text import one_hot

In [87]:
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [3]:
logistic=LogisticRegression()

In [23]:
tokenizer=ToktokTokenizer()

In [10]:
data=pd.read_csv('IMDB Dataset.csv')

In [11]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [17]:
pd.DataFrame(np.unique(data['sentiment'],return_counts=True))

<IPython.core.display.Javascript object>

,0,1
0,negative,positive
1,25000,25000


In [30]:
def remove_html(text):
    soup=BeautifulSoup(text,'html.parser')
    return soup.get_text()
    
def remove_square_brackets(text):
    return re.sub('\[[^]]*\]','',text)

def remove_special_chars(text):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

def denoise(text):
    text=remove_html(text)
    text=remove_square_brackets(text)
    text=remove_special_chars(text)
    return text


data['review']=data['review'].apply(denoise)

    
    
    
    
    
    
remove_html('<Html><Head><Body>Rishav</Body></Head></Html>')
remove_square_brackets('My name [is] Rishav ]')

'My name  Rishav ]'

In [31]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [32]:
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text
data['review']=data['review'].apply(simple_stemmer)

In [33]:
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl product the film techniqu is ve...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there a famili where a littl boy jake th...,negative
4,petter mattei love in the time of money is a v...,positive


In [48]:
def remove_stopwords(text):
    text=text.split()
    filtered=[tok.lower() for tok in text if tok.lower() not in stopwords.words('english')]
    filtered=' '.join(filtered)
    return filtered

In [49]:
data['review']=data['review'].apply(remove_stopwords)

In [50]:
data.head()

,review,sentiment
0,one review ha mention watch 1 oz episod youll ...,positive
1,wonder littl product film techniqu veri unassu...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic famili littl boy jake think zombi hi clo...,negative
4,petter mattei love time money visual stun film...,positive


In [72]:
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

In [73]:
cv_train_reviews=data.review[:40000]
cv_train_sentiment=data.sentiment[:40000]
cv_test_reviews=data.review[40000:]
cv_test_sentiment=data.sentiment[40000:]
cv_train_reviews.shape,cv_train_reviews.shape,cv_test_reviews.shape,cv_train_sentiment.shape

((40000,), (40000,), (10000,), (40000,))

In [74]:
cv_train_reviews=cv.fit_transform(cv_train_reviews)
cv_test_reviews=cv.transform(cv_test_reviews)

In [75]:
cv_train_reviews.shape,cv_test_reviews.shape

((40000, 6208633), (10000, 6208633))

In [76]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
tv_train_reviews=data.review[:40000]
tv_train_sentiment=data.sentiment[:40000]
tv_test_reviews=data.review[40000:]
tv_test_sentiment=data.sentiment[40000:]
tv_train_reviews.shape,tv_train_reviews.shape,tv_test_reviews.shape,tv_train_sentiment.shape


((40000,), (40000,), (10000,), (40000,))

In [77]:
tv_train_reviews=cv.fit_transform(tv_train_reviews)
tv_test_reviews=cv.transform(tv_test_reviews)
tv_train_reviews.shape,tv_test_reviews.shape

((40000, 6208633), (10000, 6208633))

In [78]:
lr=LogisticRegression()
lr_cv=lr.fit(cv_train_reviews,cv_train_sentiment)
lr_cv.score(cv_train_reviews,cv_train_sentiment),lr_cv.score(cv_test_reviews,cv_test_sentiment)

(0.996275, 0.5981)

In [79]:
lr=LogisticRegression()
lr_tv=lr.fit(tv_train_reviews,tv_train_sentiment)
lr_tv.score(tv_train_reviews,tv_train_sentiment),lr_tv.score(tv_test_reviews,tv_test_sentiment)

(0.996275, 0.5981)

In [97]:
onehot_rep=[one_hot(words,10000) for words in cv_train_reviews]

AttributeError: lower not found

In [94]:
model=Sequential()
model.add(Dense(input_dim=6208633,units=128,activation='sigmoid'))
model.add(Dense(units=64))
model.compile(loss='binary_crossentropy',optimizer='adam')

ResourceExhaustedError: OOM when allocating tensor with shape[6208633,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [92]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               5120128   
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
Total params: 5,128,384
Trainable params: 5,128,384
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.fit(cv_train_reviews, cv_train_sentiment, batch_size=32, epochs=10)

ValueError: Error when checking input: expected dense_6_input to have shape (40000,) but got array with shape (6208633,)